# **Hopfield Network with MNIST**

「MNIST（手書き文字データ）をネットワークで記憶→MNISTの一部クラスを選択→ノイズ付加して→ネットワークで復元」の一連のホップフィールドネットワークの動作を試すデモコードになっています。ランタイムはCPUでOKです。

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from skimage.filters import threshold_mean
from keras.datasets import mnist
import matplotlib.cm as cm
from tqdm import tqdm

class HopfieldNetwork(object):
    def train_weights(self, train_data):
        print("Start to train weights...")
        num_data =  len(train_data)
        self.num_neuron = train_data[0].shape[0]

        # initialize weights
        W = np.zeros((self.num_neuron, self.num_neuron))
        rho = np.sum([np.sum(t) for t in train_data]) / (num_data*self.num_neuron)

        # Hebb rule
        for i in tqdm(range(num_data)):
            t = train_data[i] - rho
            W += np.outer(t, t)

        # Make diagonal element of W into 0
        diagW = np.diag(np.diag(W))
        W = W - diagW
        W /= num_data

        self.W = W

    def predict(self, data, num_iter=20, threshold=0, asyn=False):
        print("Start to predict...")
        self.num_iter = num_iter
        self.threshold = threshold
        self.asyn = asyn

        # Copy to avoid call by reference
        copied_data = np.copy(data)

        # Define predict list
        predicted = []
        for i in tqdm(range(len(data))):
            predicted.append(self._run(copied_data[i]))
        return predicted

    def _run(self, init_s):
        if self.asyn==False:
            """
            Synchronous update
            """
            # Compute initial state energy
            s = init_s

            e = self.energy(s)

            # Iteration
            for i in range(self.num_iter):
                # Update s
                s = np.sign(self.W @ s - self.threshold)
                # Compute new state energy
                e_new = self.energy(s)

                # s is converged
                if e == e_new:
                    return s
                # Update energy
                e = e_new
            return s
        else:
            """
            Asynchronous update
            """
            # Compute initial state energy
            s = init_s
            e = self.energy(s)

            # Iteration
            for i in range(self.num_iter):
                for j in range(100):
                    # Select random neuron
                    idx = np.random.randint(0, self.num_neuron)
                    # Update s
                    s[idx] = np.sign(self.W[idx].T @ s - self.threshold)

                # Compute new state energy
                e_new = self.energy(s)

                # s is converged
                if e == e_new:
                    return s
                # Update energy
                e = e_new
            return s


    def energy(self, s):
        return -0.5 * s @ self.W @ s + np.sum(s * self.threshold)

    def plot_weights(self):
        plt.figure(figsize=(6, 5))
        W_norm = (self.W - np.min(self.W)) / (np.max(self.W) - np.min(self.W))
        w_mat = plt.imshow(self.W, cmap=cm.coolwarm)
        #w_mat = plt.imshow(W_norm, cmap=cm.coolwarm)
        plt.colorbar(w_mat)
        plt.title("Network Weights")
        plt.tight_layout()
        print("Saving network weights matrix...")
        plt.savefig("weights_mnist.png")
        print("Weights plot saved as weights.png.")
        #plt.show()


def reshape(data):
    dim = int(np.sqrt(len(data)))
    data = np.reshape(data, (dim, dim))
    return data

def plot(data, test, predicted, figsize=(3, 3)):
    data = [reshape(d) for d in data]
    test = [reshape(d) for d in test]
    predicted = [reshape(d) for d in predicted]

    fig, axarr = plt.subplots(len(data), 3, figsize=figsize)
    for i in range(len(data)):
        if i==0:
            axarr[i, 0].set_title('Train data')
            axarr[i, 1].set_title("Input data")
            axarr[i, 2].set_title('Output data')

        axarr[i, 0].imshow(data[i], cmap='gray')
        axarr[i, 0].axis('off')
        axarr[i, 1].imshow(test[i], cmap='gray')
        axarr[i, 1].axis('off')
        axarr[i, 2].imshow(predicted[i], cmap='gray')
        axarr[i, 2].axis('off')

    plt.tight_layout()
    plt.savefig("result_mnist.png")
    #plt.show()

def preprocessing(img):
    w, h = img.shape
    # Thresholding
    thresh = threshold_mean(img)
    binary = img > thresh
    shift = 2*(binary*1)-1 # Boolian to int

    # Reshape
    flatten = np.reshape(shift, (w*h))
    return flatten

def add_noise_to_mnist(image, noise_type='gaussian', noise_param=0.1):
    """
    画像にノイズを追加する関数

    Parameters:
    image: 元の画像データ (28x28 numpy array)
    noise_type: ノイズの種類 ('gaussian', 'salt_pepper', 'random_flip')
    noise_param: ノイズのパラメータ
                - gaussian: 標準偏差
                - salt_pepper: ノイズを加える画素の割合
                - random_flip: 反転させる画素の割合
    """
    noisy_image = np.copy(image)

    if noise_type == 'gaussian':
        # ガウシアンノイズ（スケールを調整）
        noise = np.random.normal(0, noise_param * 255, image.shape)  # ノイズのスケールを255に合わせる
        noisy_image = image + noise
        noisy_image = np.clip(noisy_image, 0, 255)  # 値を0-255の範囲に収める

    elif noise_type == 'salt_pepper':
        # Salt & Pepperノイズ
        mask = np.random.random(image.shape) < noise_param
        noisy_image[mask] = np.random.choice([0, 255], mask.sum())

    elif noise_type == 'random_flip':
        # ランダムな画素の反転
        mask = np.random.random(image.shape) < noise_param
        noisy_image[mask] = 255 - image[mask]

    return noisy_image.astype(np.uint8)


def main():
    # Load data
    (x_train, y_train), (_, _ )= mnist.load_data()
    data = []
    for i in range(3):
        xi = x_train[y_train==i]
        data.append(xi[0])

    # Preprocessing
    print("Start to data preprocessing...")
    data = [preprocessing(d) for d in data]

    # Create Hopfield Network Model
    model = HopfieldNetwork()
    model.train_weights(data)

    # テストデータの準備（ノイズ付き）
    test = []
    for i in range(3):
        xi = x_train[y_train==i]
        # オリジナルの画像にノイズを追加
        sample_number=0 # 0:original, 1:not used in train
        noisy_image = add_noise_to_mnist(
            xi[sample_number],
            noise_type='random_flip',  # 'gaussian', 'salt_pepper', 'random_flip'から選択
            noise_param=0.4        # ノイズの強さを調整
        )
        test.append(noisy_image)

    """
    # Make test datalist
    test = []
    for i in range(3):
        xi = x_train[y_train==i]
        test.append(xi[1])
    """
    test = [preprocessing(d) for d in test]

    predicted = model.predict(test, threshold=50, asyn=True)

    #print("Show prediction results...")
    plot(data, test, predicted, figsize=(5, 5))
    #print("Show network weights matrix...")
    model.plot_weights()

main()


Thanks to

*   https://github.com/shutakahama/Hopfield_Network
*   https://github.com/ethan-iai/hopfield-torch
*   https://github.com/takyamamoto/Hopfield-Network
*   https://github.com/kencyke/hopfield-mnist
*   https://www.kaggle.com/code/mrumuly/hopfield-network-mnist
*   https://github.com/grinvolod13/mnist-hopfield